In [1]:
import os
# from diffusers import DiffusionPipeline, StableDiffusionPipeline,StableDiffusionXLPipeline, StableDiffusionXLImg2ImgPipeline
import torch
from pytorch_lightning import seed_everything


from _pipeline import StableDiffusionXLPipeline, retrieve_timesteps
from models import Autoencoder
seed_everything(0)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
os.environ['http_proxy'] = 'http://127.0.0.1:7890'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
os.environ['https_proxy'] = 'http://127.0.0.1:7890'

2024-08-19 03:51:28.185420: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Seed set to 0


In [2]:
from omegaconf import OmegaConf

config = OmegaConf.load("/root/workspace/wht/multimodal_brain/src/tasks/img_generation/configs/base.yaml")

model = {}
for k,v in config['models'].items():
    if k == 'generation':
        model[k] = StableDiffusionXLPipeline.from_pretrained(pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, use_safetensors=True, variant="fp16").to("cuda")
    else:
        model[k] = globals()[v['name']](**v['args'])

Couldn't connect to the Hub: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /api/models/stabilityai/stable-diffusion-xl-base-1.0 (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fac4c9283a0>: Failed to establish a new connection: [Errno 111] Connection refused')))"), '(Request ID: 95fc66c4-b80d-444f-a0ea-43427ff8d5ca)').
Will try to load from local cache.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:
print(model['generation'].unet)
print(model['generation'].vae)
sd_model = model['generation']

UNet2DConditionModel(
  (conv_in): Conv2d(4, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (time_proj): Timesteps()
  (time_embedding): TimestepEmbedding(
    (linear_1): Linear(in_features=320, out_features=1280, bias=True)
    (act): SiLU()
    (linear_2): Linear(in_features=1280, out_features=1280, bias=True)
  )
  (add_time_proj): Timesteps()
  (add_embedding): TimestepEmbedding(
    (linear_1): Linear(in_features=2816, out_features=1280, bias=True)
    (act): SiLU()
    (linear_2): Linear(in_features=1280, out_features=1280, bias=True)
  )
  (down_blocks): ModuleList(
    (0): DownBlock2D(
      (resnets): ModuleList(
        (0-1): 2 x ResnetBlock2D(
          (norm1): GroupNorm(32, 320, eps=1e-05, affine=True)
          (conv1): Conv2d(320, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (time_emb_proj): Linear(in_features=1280, out_features=320, bias=True)
          (norm2): GroupNorm(32, 320, eps=1e-05, affine=True)
          (dropout): Dropout(p=

In [4]:
num_inference_steps = 50
device = sd_model._execution_device
timesteps = None
sigmas = None

prompt = "a photo of a cat on mars"

timesteps, num_inference_steps = retrieve_timesteps(
            sd_model.scheduler, num_inference_steps, device, timesteps, sigmas
        )
print(timesteps,num_inference_steps)

batch_size = 1
num_images_per_prompt = 1
height = 1024
width = 1024
generator = None
latents =None
torch.float16
num_channels_latents = sd_model.unet.config.in_channels
prompt_2 = None
do_classifier_free_guidance = True
negative_prompt = None
negative_prompt_2 = None
prompt_embeds = None
negative_prompt_embeds = None
pooled_prompt_embeds = None
negative_pooled_prompt_embeds = None
lora_scale = None
clip_skip = None
do_classifier_free_guidance = True
eta = 0.0
output_type = 'pil'
return_dict = False
guidance_scale = 5.0
guidance_rescale = 0.0
clip_skip = None
cross_attention_kwargs = None
denoising_end = 1.0

sd_model._guidance_scale = guidance_scale
sd_model._guidance_rescale = guidance_rescale
sd_model._clip_skip = clip_skip
sd_model._cross_attention_kwargs = cross_attention_kwargs
sd_model._denoising_end = denoising_end
sd_model._interrupt = False

(
    prompt_embeds,
    negative_prompt_embeds,
    pooled_prompt_embeds,
    negative_pooled_prompt_embeds,
) = sd_model.encode_prompt(
        prompt=prompt,
        prompt_2=prompt_2,
        device=device,
        num_images_per_prompt=num_images_per_prompt,
        do_classifier_free_guidance=do_classifier_free_guidance,
        negative_prompt=negative_prompt,
        negative_prompt_2=negative_prompt_2,
        prompt_embeds=prompt_embeds,
        negative_prompt_embeds=negative_prompt_embeds,
        pooled_prompt_embeds=pooled_prompt_embeds,
        negative_pooled_prompt_embeds=negative_pooled_prompt_embeds,
        lora_scale=lora_scale,
        clip_skip=clip_skip,
    )
latents = sd_model.prepare_latents(
    batch_size * num_images_per_prompt,
    num_channels_latents,
    height,
    width,
    torch.float16,
    device,
    generator,
    latents,
)
extra_step_kwargs = sd_model.prepare_extra_step_kwargs(generator, eta)
add_text_embeds = pooled_prompt_embeds
text_encoder_projection_dim = sd_model.text_encoder_2.config.projection_dim
original_size =  (height, width)
crops_coords_top_left = (0, 0)
target_size =  (height, width)
add_time_ids = sd_model._get_add_time_ids(
            original_size,
            crops_coords_top_left,
            target_size,
            dtype=prompt_embeds.dtype,
            text_encoder_projection_dim=text_encoder_projection_dim,
        )
negative_add_time_ids = add_time_ids


if do_classifier_free_guidance:
    prompt_embeds = torch.cat([negative_prompt_embeds, prompt_embeds], dim=0)
    add_text_embeds = torch.cat([negative_pooled_prompt_embeds, add_text_embeds], dim=0)
    add_time_ids = torch.cat([negative_add_time_ids, add_time_ids], dim=0)

prompt_embeds = prompt_embeds.to(device)
add_text_embeds = add_text_embeds.to(device)
add_time_ids = add_time_ids.to(device).repeat(batch_size * num_images_per_prompt, 1)
num_warmup_steps = max(len(timesteps) - num_inference_steps * sd_model.scheduler.order, 0)

timestep_cond = None
sd_model._num_timesteps = len(timesteps)
# print(prompt_embeds.shape)
# print(negative_prompt_embeds.shape)
# print(pooled_prompt_embeds.shape)
# print(negative_pooled_prompt_embeds.shape)
# print(latents.shape)
# print(add_time_ids.shape)

# print(negative_prompt_embeds)
# print(prompt_embeds)
# print(sd_model.scheduler)

tensor([981., 961., 941., 921., 901., 881., 861., 841., 821., 801., 781., 761.,
        741., 721., 701., 681., 661., 641., 621., 601., 581., 561., 541., 521.,
        501., 481., 461., 441., 421., 401., 381., 361., 341., 321., 301., 281.,
        261., 241., 221., 201., 181., 161., 141., 121., 101.,  81.,  61.,  41.,
         21.,   1.], device='cuda:0') 50


In [10]:
prompt_embeds.shape
added_cond_kwargs

{'text_embeds': tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.8608, -0.2827,  1.0537,  ..., -0.5127, -1.5000,  0.3979]],
        device='cuda:0', dtype=torch.float16, grad_fn=<CatBackward0>),
 'time_ids': tensor([[1024., 1024.,    0.,    0., 1024., 1024.],
         [1024., 1024.,    0.,    0., 1024., 1024.]], device='cuda:0',
        dtype=torch.float16)}

In [5]:
with torch.no_grad():
        with sd_model.progress_bar(total=num_inference_steps) as progress_bar:
                for i, t in enumerate(timesteps):
                        latent_model_input = torch.cat([latents] * 2) if do_classifier_free_guidance else latents
                        latent_model_input = sd_model.scheduler.scale_model_input(latent_model_input, t)
                        added_cond_kwargs = {"text_embeds": add_text_embeds, "time_ids": add_time_ids}

                        noise_pred = sd_model.unet(
                                latent_model_input,
                                t,
                                encoder_hidden_states=prompt_embeds,
                                timestep_cond=timestep_cond,
                                cross_attention_kwargs=sd_model.cross_attention_kwargs,
                                added_cond_kwargs=added_cond_kwargs,
                                return_dict=False,
                        )[0]


                        if do_classifier_free_guidance:
                                noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
                                noise_pred = noise_pred_uncond + sd_model.guidance_scale * (noise_pred_text - noise_pred_uncond)

                        latents_dtype = latents.dtype
                        latents = sd_model.scheduler.step(noise_pred, t, latents, **extra_step_kwargs, return_dict=False)[0]

                        if i == len(timesteps) - 1 or ((i + 1) > num_warmup_steps and (i + 1) % sd_model.scheduler.order == 0):
                                progress_bar.update()

        needs_upcasting = sd_model.vae.dtype == torch.float16 and sd_model.vae.config.force_upcast
        if needs_upcasting:
                sd_model.upcast_vae()
                latents = latents.to(next(iter(sd_model.vae.post_quant_conv.parameters())).dtype)
        # elif latents.dtype != sd_model.vae.dtype:
        #         if torch.backends.mps.is_available():
        #                 # some platforms (eg. apple mps) misbehave due to a pytorch bug: https://github.com/pytorch/pytorch/pull/99272
        #                 sd_model.vae = sd_model.vae.to(latents.dtype)
        latents = latents / sd_model.vae.config.scaling_factor
        image = sd_model.vae.decode(latents, return_dict=False)[0]

        if needs_upcasting:
                sd_model.vae.to(dtype=torch.float16)

        image = sd_model.image_processor.postprocess(image, output_type=output_type)

        # Offload all models
        sd_model.maybe_free_model_hooks()

        if not return_dict:
                output = (image,)

  0%|          | 0/50 [00:00<?, ?it/s]

In [6]:
print(output)
output[0][0].save('results/mar_cat.png')

([<PIL.Image.Image image mode=RGB size=1024x1024 at 0x7FAC439BDE80>],)


In [ ]:
pipeline = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",torch_dtype=torch.float16, variant="fp16", use_safetensors=True
)
pipeline.to("cuda")


In [ ]:
# print(pipeline.cross_attention_kwargs)
# print(pipeline.encode_prompt)
# print(pipeline.do_classifier_free_guidance)
# print(pipeline.negative_prompt)
prompt = "a photo of an astronaut riding a horse on mars"
# prompt = ""
n_steps = 30
high_noise_frac = 1.0
output = pipeline(
    prompt=prompt,
    num_inference_steps=n_steps,
    denoising_end=high_noise_frac,
    # output_type="latent",
)

In [ ]:
print(output['images'][0])

In [ ]:
output['images'][0].save('results/mar_horse_2.jpg')

In [ ]:
# output['images'].shape #latent 1, 4, 128, 128
output['images'][0]

In [ ]:
# Encode input prompt

cross_attention_kwargs = None
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
num_images_per_prompt = 1
do_classifier_free_guidance = True

lora_scale = (
    cross_attention_kwargs.get("scale", None) if cross_attention_kwargs is not None else None
)

prompt_embeds, negative_prompt_embeds = pipeline.encode_prompt(
    prompt,
    device,
    num_images_per_prompt,
    do_classifier_free_guidance,
    negative_prompt,
    prompt_embeds=prompt_embeds,
    negative_prompt_embeds=negative_prompt_embeds,
    lora_scale=lora_scale,
    clip_skip=self.clip_skip,
)

# For classifier free guidance, we need to do two forward passes.
# Here we concatenate the unconditional and text embeddings into a single batch
# to avoid doing two forward passes
if self.do_classifier_free_guidance:
    prompt_embeds = torch.cat([negative_prompt_embeds, prompt_embeds])